<p>
<img src="https://alanattableau.files.wordpress.com/2014/12/cropped-databender4.png" align=center>
<h1 align=center>The Last Databenders</h1>
<h4 align=center>Sponsored By Gutenberg</h4>
<img src="http://www.gutenberg.org/pics/logo-144x144.png" align=center>
</p>

In [1]:
from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_9178c0c603054adab28f2ffa6291055a(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '4cccad6c139848aebd2675161c64efb3')
    hconf.set(prefix + '.username', '6a1dd27646d5462180eb41ac1b410d3b')
    hconf.set(prefix + '.password', 'PFr{4oJO9-9/ZpD^')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_9178c0c603054adab28f2ffa6291055a(name)

spark = SparkSession.builder.getOrCreate()

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initalized for you.
# The following variable contains the path to your file on your Object Storage.

In [2]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_9178c0c603054adab28f2ffa6291055a(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_00e5185dcac3063e1e316af1098498a1cacdc373','domain': {'id': 'dae6eace16224226bb3816b040f2b308'},
            'password': 'PFr{4oJO9-9/ZpD^'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

# Your data file was loaded into a StringIO object and you can process the data.
# Please read the documentation of pandas to learn more about your possibilities to load your data.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html

In [ ]:
path_1 = "swift://TheLastDatabendersCS340." + name + "/en.txt"
slang_rdd = sc.textFile(path_1)

In [3]:
stopworddd = ['a', "a's", 'able', 'about', 'above', 'according',"project","gutenberg","ebook",'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly',"the", 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', "couldn't", 'course', 'currently', 'd', 'definitely', 'described', 'despite', 'did', "didn't", 'different', 'do', 'does', "doesn't", 'doing', "don't", 'done', 'down', 'downwards', 'during', 'e', 'each', 'edu', 'eg', 'eight', 'either', 'else', 'elsewhere', 'enough', 'entirely', 'especially', 'et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'f', 'far', 'few', 'fifth', 'first', 'five', 'followed', 'following', 'follows', 'for', 'former', 'formerly', 'forth', 'four', 'from', 'further', 'furthermore', 'g', 'get', 'gets', 'getting', 'given', 'gives', 'go', 'goes', 'going',"things","electronic work","electronic","work",'gone', 'got', 'gotten', 'greetings', 'h', 'had', "hadn't", 'happens', 'hardly', 'has', "hasn't", 'have', "haven't", 'having', 'he', "he's", 'hello', 'help', 'hence', 'her', 'here', "here's", 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit', 'however', 'i', "i'd", "i'll", "i'm", "i've", 'ie', 'if', 'ignored', 'immediate', 'in', 'inasmuch', 'inc', 'indeed', 'indicate', 'indicated', 'indicates', 'inner', 'insofar', 'instead', 'into', 'inward', 'is', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kept', 'know', 'knows', 'known', 'l', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', "let's", 'like', 'liked', 'likely', 'little', 'look', 'looking', 'looks', 'ltd', 'm', 'mainly', 'many', 'may', 'maybe', 'me', 'mean', 'meanwhile', 'merely', 'might', 'more', 'moreover', 'most', 'mostly', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely', 'nd', 'near', 'nearly', 'necessary', 'need', 'needs', 'neither', 'never', 'nevertheless', 'new', 'next', 'nine', 'no', 'nobody', 'non', 'none', 'noone', 'nor', 'normally', 'not', 'nothing', 'novel', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'p', 'particular', 'particularly', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provides', 'q', 'que', 'quite', 'qv', 'r', 'rather', 'rd', 're', 'really', 'reasonably', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 's', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', 'she', 'should', "shouldn't", 'since', 'six', 'so', 'some', 'somebody', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', 't', "t's", 'take', 'taken', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', "that's", 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', "there's", 'thereafter', 'thereby',"place","moment", 'therefore', 'therein', 'theres', 'thereupon', 'these', 'they', "they'd", "they'll", "they're", "they've", 'think', 'third', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless', 'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'uucp', 'v', 'value', 'various', 'very', 'via', 'viz', 'vs', 'w', 'want', 'wants', 'was', "wasn't", 'way', 'we', "we'd", "we'll", "we're", "we've", 'welcome', 'well', 'went', 'were', "weren't", 'what', "what's", 'whatever', 'when', 'whence', 'whenever', 'where', "where's", 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', "who's", 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', "won't", 'wonder', 'would', 'would', "wouldn't", 'x', 'y', 'yes', 'yet', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 'z', 'zero',"made","time","thing","man","hand","men","word","thought","day","electronic work","found"]
adjectives = ['able', 'bad', 'best',"back", 'better', 'big', 'black', 'certain', 'clear', 'early', 'easy', 'free', 'full', 'good', 'great', 'hard', 'high', 'important', 'large', 'late', 'little', 'local', 'long', 'low', 'major', 'new', 'old', 'only', 'other', 'political', 'possible', 'public', 'real', 'recent', 'right', 'small', 'social', 'special', 'strong', 'sure', 'true', 'white', 'whole', 'young']
common_verbs = ['say', 'make', 'go', 'take', 'come', 'see', 'know', 'get', 'give', 'find', 'think', 'tell', 'become', 'show', 'leave', 'feel', 'put', 'bring', 'begin', 'keep', 'hold', 'write', 'stand', 'hear', 'let', 'mean', 'set', 'meet', 'run', 'pay', 'sit', 'speak', 'lie', 'lead', 'read', 'grow', 'lose', 'fall', 'send', 'build', 'draw', 'break', 'spend', 'cut', 'rise', 'drive', 'buy', 'wear', 'choose', 'said', 'made', 'went', 'took', 'came', 'saw', 'knew', 'got', 'gavefound', 'thought', 'told', 'became', 'showed', 'left', 'felt', 'put', 'brought', 'began', 'kept', 'held', 'wrote', 'stood', 'heard', 'let', 'meant', 'set', 'met', 'ran', 'paid', 'sat', 'spoke', 'lay', 'led', 'read', 'grew', 'lost', 'fell', 'sent', 'built', 'drew', 'broke', 'spent', 'cut', 'rose', 'drove', 'bought', 'wore', 'chose', 'gone', 'taken', 'seen', 'known', 'given', 'shown', 'written', 'spoken', 'lain', 'grown', 'fallen', 'drawn', 'broken', 'risen', 'driven', 'worn', 'chosen', 'wanted', 'used', 'worked', 'called', 'tried', 'asked', 'needed', 'seemed', 'helped', 'played', 'moved', 'lived', 'believed', 'happened', 'included', 'continued', 'changed', 'watched', 'followed', 'stopped', 'created', 'opened', 'walked', 'offered', 'remembered', 'appeared', 'served', 'died', 'stayed', 'reached', 'killed', 'raised', 'passed', 'decided', 'returned', 'explained', 'hoped', 'carried', 'received', 'agreed', 'covered', 'caused', 'closed', 'increased', 'reduced', 'noted', 'entered', 'shared', 'saved', 'protected', 'occurred', 'accepted', 'determined', 'prepared', 'argued', 'recognized', 'indicated', 'arrived', 'answered', 'compared', 'acted', 'studied', 'removed', 'sounded', 'formed', 'visited', 'avoided', 'imagined', 'finished', 'responded', 'maintained', 'contained', 'applied', 'treated', 'affected', 'worried', 'mentioned', 'improved', 'signed', 'existed', 'noticed', 'travelled', 'prevented', 'admitted', 'suffered', 'published', 'counted', 'achieved', 'announced', 'touched', 'attended', 'defined', 'introduced']
final_stopwords = stopworddd + adjectives + common_verbs

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.sql import Row
rdd = sc.textFile("main_Categories.txt")
def TrainData(genre):
    interim = rdd.map(lambda line: (1.0 if line.split("\t")[0] == genre else 0.0, line.split("\t")[1]))
    interim = interim.map(lambda t: Row(label=t[0], text=t[1]))
    training = sqlContext.createDataFrame(interim).cache()
    tokenizer = Tokenizer(inputCol="text", outputCol="word")
    stopwordremover = StopWordsRemover(inputCol="word", outputCol="words", stopWords=final_stopwords)
    hashingTF = HashingTF(inputCol=stopwordremover.getOutputCol(), outputCol="features")
    lr = LogisticRegression(maxIter=10, regParam=0.01)
    pipeline = Pipeline(stages=[tokenizer, stopwordremover, hashingTF, lr])

    # Fit the pipeline to training documents.
    model = pipeline.fit(training)
    return model

In [5]:
model = TrainData("Children")

In [6]:
############## TEST SECTION ####################

In [7]:
testlist = ["test_1_F.txt","test_2_F.txt","test_3_F.txt","test_4_F.txt","test_5_F.txt","test_6_F.txt","test_1_N.txt","test_1_T.txt","test_2_T.txt","test_3_T.txt","test_4_T.txt","test_5_T.txt","test_6_T.txt"]

In [8]:
from pyspark.sql import Row
def test(book):  
    data_1 = get_object_storage_file_with_credentials_9178c0c603054adab28f2ffa6291055a('TheLastDatabendersCS340', book)
    test = data_1.read()
    testDF = sqlContext.createDataFrame([Row(text=test)])
    result = model.transform(testDF) 
    prediction = result.select("prediction")
    prediction = prediction.rdd.map(tuple).collect()[0][0]
    test_rdd = sc.parallelize(test.replace("\r"," ").replace("\n"," ").split(" "))
    test_rdd = test_rdd.map(lambda line: line.lower())
    test_rdd = test_rdd.filter(lambda line: line != '')
    slang_match = len(slang_rdd.intersection(test_rdd).collect())
    if slang_match == 0:
        if prediction == 1.0: 
            print "It is appropriate for children."
        else: 
            print "It is inappropriate for children. It contains sexuality and/or violence."
    elif slang_match < 7:
        print "It contains a few slang words."    
        if prediction == 1.0:
            print "It is recommended to be read under parental supervision."    
        else:
            print "It is inappropriate for children. It contains sexuality and/or violence."        
    else:
        print "It is inappropriate for children. It may contain intense violence, sexual content and/or strong language."
    

In [12]:
test("test_2_F.txt")

It contains a few slang words.
It is inappropriate for children. It contains sexuality and/or violence.
